In [ ]:
import pandas as pd
import json

# Load file Excel
file_path = "../../datasets/xlsx/Dataset Labeling Chatbot.xlsx"
blast_df = pd.read_excel(file_path, sheet_name="Blast")
tungro_df = pd.read_excel(file_path, sheet_name="Tungro")
bacterial_blight_df = pd.read_excel(file_path, sheet_name="BacterialBlight")
brown_spot_df = pd.read_excel(file_path, sheet_name="BrownSpot")
leaf_scald_df = pd.read_excel(file_path, sheet_name="LeafScald")

# Tambahkan kolom 'penyakit' untuk tiap dataframe
blast_df["penyakit"] = "Blast"
tungro_df["penyakit"] = "Tungro"
bacterial_blight_df["penyakit"] = "Bacterial Blight"
brown_spot_df["penyakit"] = "Brown Spot"
leaf_scald_df["penyakit"] = "Leaf Scald"

# Gabungkan semua
chatbot_df = pd.concat(
    [blast_df, tungro_df, bacterial_blight_df, brown_spot_df, leaf_scald_df],
    ignore_index=True,
)

# Ambil kolom yang dibutuhkan
chatbot_data = chatbot_df[["penyakit", "prompt", "response"]].dropna()

# Format ulang prompt dengan prefix <instruction>
chatbot_data["prompt"] = chatbot_data.apply(
    lambda row: f"<instruction>\nPenyakit: {row['penyakit']}\nPertanyaan: {row['prompt']}\nJawaban:",
    axis=1,
)

# Simpan ke JSONL
with open("chatbot_data_instruction.jsonl", "w", encoding="utf-8") as f:
    for _, row in chatbot_data.iterrows():
        json.dump(
            {"prompt": row["prompt"], "response": row["response"]},
            f,
            ensure_ascii=False,
        )
        f.write("\n")

In [14]:
import pandas as pd
import json

# === Load file Excel ===
file_path = "../../datasets/xlsx/Dataset Labeling Chatbot.xlsx"
blast_df = pd.read_excel(file_path, sheet_name="Blast")
tungro_df = pd.read_excel(file_path, sheet_name="Tungro")
bacterial_blight_df = pd.read_excel(file_path, sheet_name="BacterialBlight")
brown_spot_df = pd.read_excel(file_path, sheet_name="BrownSpot")
leaf_scald_df = pd.read_excel(file_path, sheet_name="LeafScald")

# === Tambahkan label penyakit ===
blast_df["penyakit"] = "Blast"
tungro_df["penyakit"] = "Tungro"
bacterial_blight_df["penyakit"] = "Bacterial Blight"
brown_spot_df["penyakit"] = "Brown Spot"
leaf_scald_df["penyakit"] = "Leaf Scald"

# === Gabungkan semua data ===
chatbot_df = pd.concat(
    [blast_df, tungro_df, bacterial_blight_df, brown_spot_df, leaf_scald_df],
    ignore_index=True,
)

chatbot_df = chatbot_df[["penyakit", "prompt"]].dropna()

# === Template jawaban per penyakit ===
template_responses = {
    "Blast": (
        "Penyebab: Penyakit blast disebabkan oleh jamur Magnaporthe oryzae.\n"
        "Gejala: Bercak berbentuk belah ketupat berwarna cokelat keabu-abuan muncul di daun padi. "
        "Pada serangan berat, bercak menyebar luas dan daun mengering.\n"
        "Dampak: Infeksi berat dapat menurunkan hasil panen hingga 70%."
    ),
    "Tungro": (
        "Penyebab: Penyakit tungro disebabkan oleh kombinasi dua virus, RTSV dan RTBV, yang ditularkan melalui wereng hijau.\n"
        "Gejala: Daun menguning, kaku, dan pertumbuhan tanaman terhambat. Daun bisa menggulung ke bawah.\n"
        "Dampak: Kehilangan hasil panen bisa mencapai 100% jika infeksi terjadi pada fase awal."
    ),
    "Bacterial Blight": (
        "Penyebab: Penyakit ini disebabkan oleh bakteri Xanthomonas oryzae pv. oryzae.\n"
        "Gejala: Daun menunjukkan bercak putih kekuningan yang memanjang dari tepi daun. Daun bisa layu dan mati.\n"
        "Dampak: Penurunan hasil panen hingga 60% pada kasus berat."
    ),
    "Brown Spot": (
        "Penyebab: Penyakit ini disebabkan oleh jamur Bipolaris oryzae.\n"
        "Gejala: Muncul bercak bulat berwarna cokelat gelap dengan tepi cokelat kemerahan pada daun.\n"
        "Dampak: Dapat menurunkan kualitas dan kuantitas gabah jika tidak dikendalikan."
    ),
    "Leaf Scald": (
        "Penyebab: Penyakit leaf scald disebabkan oleh jamur Microdochium oryzae.\n"
        "Gejala: Daun menunjukkan bercak cokelat tua yang berkembang dari ujung dan tepi daun, menyerupai bekas luka.\n"
        "Dampak: Infeksi berat bisa menyebabkan daun mati dan menurunkan hasil panen."
    ),
}

# === Format dan simpan ulang ===
output_path = "chatbot_data_cleaned.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in chatbot_df.iterrows():
        formatted_prompt = f"<instruction>\nPenyakit: {row['penyakit']}\nPertanyaan: {row['prompt']}\nJawaban:"
        response = template_responses[row["penyakit"]]
        json.dump(
            {"prompt": formatted_prompt, "response": response}, f, ensure_ascii=False
        )
        f.write("\n")

In [2]:
# Siapkan data untuk RAG
rag_data = []

for _, row in chatbot_df.iterrows():
    if pd.notnull(row["response"]):
        doc = {
            "text": row["response"],
            "metadata": {
                "penyakit": row.get("Penyakit", "Unknown"),
                "topik": row.get("topik", "Unknown"),
                "sumber": row.get("sumber", ""),
            },
        }
        rag_data.append(doc)

# Simpan dalam format JSONL
with open("rag_documents.jsonl", "w", encoding="utf-8") as f:
    for item in rag_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

In [1]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)

2025-06-13 02:13:17.099050: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-13 02:13:17.109526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749751997.123688   10043 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749751997.128056   10043 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749751997.137927   10043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:


# Load dataset JSONL
dataset = load_dataset("json", data_files="chatbot_data_reformat.jsonl", split="train")

# Gabungkan prompt dan response jadi satu kolom 'text' untuk training
dataset = dataset.map(lambda x: {"text": f"{x['prompt']} {x['response']}"})

# Load model & tokenizer
model_name = "cahya/indochat-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Tambah pad token jika belum ada
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


# Tokenisasi
def tokenize_function(example):
    return tokenizer(
        example["text"], truncation=True, padding="max_length", max_length=128
    )


tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./indochat-chatbot",
    evaluation_strategy="no",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    report_to="none",
    overwrite_output_dir=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

/home/ruvne/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_10043/3809624487.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,3.010900
20,2.775600
30,2.604700
40,2.619700
50,2.451300
60,2.345000
70,2.332400
80,2.489200
90,2.382100
100,2.219300


TrainOutput(global_step=282, training_loss=2.2950226237587894, metrics={'train_runtime': 263.899, 'train_samples_per_second': 4.263, 'train_steps_per_second': 1.069, 'total_flos': 261197070336000.0, 'train_loss': 2.2950226237587894, 'epoch': 3.0})

In [3]:
trainer.save_model("indochat-chatbot-finetuned")
tokenizer.save_pretrained("indochat-chatbot-finetuned")

('indochat-chatbot-finetuned/tokenizer_config.json',
 'indochat-chatbot-finetuned/special_tokens_map.json',
 'indochat-chatbot-finetuned/vocab.json',
 'indochat-chatbot-finetuned/merges.txt',
 'indochat-chatbot-finetuned/added_tokens.json',
 'indochat-chatbot-finetuned/tokenizer.json')

In [5]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("indochat-chatbot-finetuned")
chatbot = pipeline(
    "text-generation", model="indochat-chatbot-finetuned", tokenizer=tokenizer
)
chatbot(
    "<instruction>Penyakit: Blast\nPertanyaan: Apa penyebabnya?.\nJawaban:",
    max_new_tokens=85,
    temperature=0.5,
    top_p=0.85,
    repetition_penalty=1.4,
    no_repeat_ngram_size=5,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
)

Device set to use cuda:0


[{'generated_text': '<instruction>Penyakit: Blast\nPertanyaan: Apa penyebabnya?.\nJawaban: Penyakit blast disebabkan oleh jamur yang disebut Chlamydia pulchella. Jamur ini menghasilkan spora dan dapat menginfeksi tanaman padi atau gulma lain, serta menyebabkan kerusakan pada benih akibat pelepasan glikosida ke dalam air nutrisi saat proses fotosintesis berlangsung. Spora kemudian bergerak melalui udara (airborne) untuk menyebarkan penyakit dari satu titik infeksi menuju jaringan sehat lainnya di seluruh area tanam sehingga merusak hasil panen secara signifikan hingga menjadi gagal tumbuh total tanpa gejala'}]